In [45]:
import pandas as pd
import numpy as np
import datetime
import itertools
import matplotlib.pyplot as plt
from IPython.display import display
%matplotlib inline

from bravado.client import SwaggerClient
from bravado.exception import HTTPError

%load_ext autoreload

%autoreload 2
import helper
from Station import Station
from Query import Query

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Import client modules
These let us interface with the Swagger Client and pull info from EVE's API

In [2]:
# Lookup type and region info form the SDE
sde_client = SwaggerClient.from_url("http://evekit-sde.orbital.enterprises/latest/swagger.json",
                                   config={'ues_models': False, 'validate_responses' : False})

# We need a SwaggerClient for the market data colleciton service
mdc_client = SwaggerClient.from_url("https://evekit-market.orbital.enterprises//swagger",
                                   config={'use_models': False, 'validate_responses' : False})

### Specify stations and items of interest

In [3]:
# Choose items and stations of interest
# Station Names
names_station = ['Jita IV - Moon 4 - Caldari navy Assembly Plant', 
                 'Amarr VIII (Oris) - Emperor Family Academy']
names_station_shorthand = ['Jita 4-4', 'Amarr']
dict_names = dict(zip(names_station, names_station_shorthand))
# Region names
names_region = ['The Forge', 'Domain']
# Item names
names_items = ["Biocells","Construction Blocks","Consumer Electronics"]

# Date range of interest (generally just today's date for simple trade find)
date = datetime.date.today() - datetime.timedelta(days = 1)

In [4]:
stations = {key: Station(sde_client, key, value) for key, value in dict_names.items()}

In [5]:
for key, station in stations.items():
    print(station.get_station_name())

Jita IV - Moon 4 - Caldari navy Assembly Plant
Amarr VIII (Oris) - Emperor Family Academy


In [6]:
for key, station in stations.items():
    print(station)
    print(f"Station Name     : %s"%station.get_station_name())
    print(f"Station Shorthand: %s"%station.get_station_shorthand())
    print(f"Station ID       : %d"%station.get_station_id())
    print(f"Station Region ID: %s"%station.get_region_id())
    print()

Station Name     : Jita IV - Moon 4 - Caldari navy Assembly Plant
Station Shorthand: Jita 4-4
Station ID       : 60003760
Station Region ID: 10000002

Station Name     : Amarr VIII (Oris) - Emperor Family Academy
Station Shorthand: Amarr
Station ID       : 60008494
Station Region ID: 10000043



In [7]:
# Look up the relative IDs for the names above
dict_station = {x: helper.get_station_id(sde_client, x) for x in names_station}
dict_region  = {x: helper.get_region_id(sde_client, x) for x in names_region}
dict_items   = {x: helper.get_type_id(sde_client, x) for x in names_items}

In [ ]:
df_internal_spread = helper.construct_internal_spread(mdc_client, dict_items, stations, date)

In [46]:
query = Query(sde_client, mdc_client, dict_items, stations, date)

In [47]:
query.get_master_df().head()

,Items,Jita 4-4: buys,Jita 4-4: sells,Amarr: buys,Amarr: sells,Jita 4-4: Internal Spread,Amarr: Internal Spread
0,Biocells,9601.00,10232.00,8201.24,10498.23,631.00,2296.99
1,Construction Blocks,12705.00,12999.97,9726.08,12999.94,294.97,3273.86
2,Consumer Electronics,13101.74,14456.90,13200.00,14168.99,1355.16,968.99


In [43]:
df_comparison_spread = helper.construct_comparison_spread(mdc_client, dict_items, stations, date)

KeyError: ('Items', 'occurred at index 0')

In [ ]:
df_comparison_spread.head()

In [ ]:
# Constuct the master dataframe to house everything
df_internal_spread = pd.DataFrame(list(dict_items.keys()))
df_internal_spread.columns = ['Items']
df_internal_spread.head()

In [ ]:
df_internal_spread = helper.construct_internal_spread(df_internal_spread, stations, dict_items)

### Testing Grounds
Testing code

In [ ]:
# Test constructing a cross-product of dict key/values
test_dict = {"a": 0, "b": 1, "c": 2}
test = itertools.product(*test_dict.keys())
for item in test:
    print(item)